### Getting differential operators

In [3]:
import torch
import torch.nn as nn
from torch.autograd import grad
import torch.nn.functional as F
import math

torch.set_printoptions(precision=8)

In [35]:
x = torch.ones(2, 2, requires_grad=True)
v = x + 2
y = v ** 2

dy_dx = grad(outputs=y, inputs=x, grad_outputs=torch.ones_like(y))
print(f'dy/dx:\n {dy_dx}')

dv_dx = grad(outputs=v, inputs=x, grad_outputs=torch.ones_like(v))
print(f'dv/dx:\n {dv_dx}')

dy/dx:
 (tensor([[6., 6.],
        [6., 6.]]),)
dv/dx:
 (tensor([[1., 1.],
        [1., 1.]]),)


In [4]:
class MyFunction(nn.Module):
    def __init__(self,input_size=2,output_size=1):
        super(MyFunction, self).__init__()
        self.linear1 = nn.Linear(input_size,5)
        self.linear2 = nn.Linear(5,5)
        self.linear3 = nn.Linear(5,output_size)

    def forward(self,x):
        z = torch.sigmoid(self.linear1(x))
        z = torch.sigmoid(self.linear2(z))
        z = torch.sigmoid(self.linear3(z))
        return z

In [5]:
func = MyFunction()

x = torch.rand(8, 2, requires_grad=True)
z = func(x)

dz_dx = grad(outputs=z, inputs=x, grad_outputs=torch.ones_like(z))[0]
print(f'dz/dx:\n {dz_dx}')

dz/dx:
 tensor([[0.00070088, 0.00038788],
        [0.00071402, 0.00037485],
        [0.00070843, 0.00038154],
        [0.00069590, 0.00039529],
        [0.00072330, 0.00035239],
        [0.00068689, 0.00039576],
        [0.00071221, 0.00037727],
        [0.00069894, 0.00038993]])


In [38]:
def testm(x):
    y = (x + 1) ** 3
    return y

In [29]:
def gradfn(model, order, xs):
    out = model(xs)
    for i in range(order):
        out = grad(outputs=out, inputs=xs, grad_outputs=torch.ones_like(out), allow_unused=True)[0]
        out.requires_grad = True
    return out

### Using torch.autograd.functional.hessian (for second derivatives)

In [9]:
def quadratic(x):
    y = 3 * x ** 3 + x ** 2 + 4
    return torch.sum(y)

In [10]:
x = torch.tensor([2,5], dtype=torch.float64)

torch.autograd.functional.hessian(quadratic, x, create_graph=True)

tensor([[38.,  0.],
        [ 0., 92.]], dtype=torch.float64, grad_fn=<ViewBackward>)

In [11]:
def batch_sec(fn, x, grads): # paralellize this function
    out = torch.zeros(x.size()[0], len(grads))

    for i, xi in enumerate(x):
        hess = torch.autograd.functional.hessian(fn, xi, create_graph=True)
        for n, dx in enumerate(grads):
            out[i,n] = hess[dx]

    return out

In [16]:
x = torch.tensor([[1, 2], [3,4], [5, 6], [7, 8]], dtype=torch.float64, requires_grad=True)

# print(batch_sec(quadratic, x, grads=[(0,0), (1,1)]))
# print(batch_sec(func, x, grads=[(0,0), (1,1)]))

### Simpler Autograd

In [32]:
def sec_der(y,x):
    y_x = grad(y, x, grad_outputs=torch.ones_like(y),
    retain_graph=True,
    create_graph=True,
    allow_unused=True)[0]

    y_xx = grad(y_x, x, grad_outputs=torch.ones_like(y_x),
    retain_graph=True,
    create_graph=True,
    allow_unused=True)[0]

    return y_xx

In [37]:
print(sec_der(quadratic(x),x))
print(sec_der(func(x.float()),x))

tensor([[ 20.,  38.],
        [ 56.,  74.],
        [ 92., 110.],
        [128., 146.]], dtype=torch.float64, grad_fn=<AddBackward0>)
tensor([[ 6.31085422e-05, -4.60458978e-05],
        [-1.19884426e-04, -5.69167169e-05],
        [-1.38242845e-04, -4.16072362e-05],
        [-8.08083278e-05, -1.79816343e-05]], dtype=torch.float64,
       grad_fn=<CopyBackwards>)
